In [5]:
import nltk
from nltk.chat.util import Chat, reflections
from tkinter import *

In [6]:
# Definir patrones y respuestas
patterns_responses = [
    (r'hello|hi|hey|hola|que tal|qué tal|buenos dias|buenos días|buenas tardes|buenas noches', ['¡Hola! ¿Cómo puedo ayudarte hoy?']),
    (r'mondongo', ['mondongo tu']),
    (r'how are you', ['I am just a bot, but I am doing fine!']),
    (r'what is your name', ['I am a chatbot created with NLTK.']),
    (r'quit|exit', ['Goodbye! Have a great day!']),
    (r'(.*) your (.*)', ['Why do you want to know about my {1}?'])
]

# Crear el chatbot
chatbot = Chat(patterns_responses, reflections)

In [9]:
# Define la función para enviar el mensaje
def send(event=None):
    global chatbot  # Asegurar que se use la instancia global de chatbot
    # Obtén el texto del EntryBox
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)

    if msg != "":
        # Mostrar el mensaje del usuario en el chat
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12))

        # Obtener la respuesta del chatbot
        response = chatbot.respond(msg.lower())
        if response:
            ChatLog.insert(END, "SerraBot: " + response + '\n\n')
        else:
            ChatLog.insert(END, "SerraBot: I'm not sure how to respond to that.\n\n")

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

# Inicializa la ventana principal de Tkinter
base = Tk()

# Configura el título de la ventana
base.title("SerraBot")

# Establece el tamaño de la ventana
base.geometry("400x500")

# Evita que el usuario cambie el tamaño de la ventana
base.resizable(width=False, height=False)

# Crear ChatLog - un widget de texto para mostrar la conversación
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial")
ChatLog.config(state=DISABLED)

# Crear una barra de desplazamiento y enlazarla al ChatLog
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

# Crear EntryBox - un widget de texto para que el usuario escriba mensajes
EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font="Arial")

# Enlazar la tecla Enter con la función send
EntryBox.bind("<Return>", send)

# Colocar todos los componentes en la ventana
scrollbar.place(x=376, y=6, height=386)
ChatLog.place(x=6, y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)

# Iniciar el bucle de eventos de Tkinter
base.mainloop()